In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [ ]:
gt_dir = "../data/tests/"
dir_name = gt_dir #+"output/"
files = os.listdir(dir_name)
files = list(sorted(filter(lambda x: x.endswith(".csv"), files)))
print(files)
data = files[-2]

df = pd.read_csv(dir_name+"/"+data)


df = df.drop(columns=["Unnamed: 12"])
df

In [ ]:
plt.plot(np.arange(len(df)), df["Yaw"], label="yaw")
plt.plot(np.arange(len(df)), (df["Actual Yaw"].to_numpy()), label="gtyaw")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

fps = 10
period = 1/fps

t = np.arange(0, len(df)/fps, period)
print(t[-1])

axes[0,0].plot(t, df["Total Offset X"], label="x")
axes[0,0].plot(t, df["Actual X"], label="gt x")

axes[0,1].plot(t, df["Total Offset Y"], label="y")
axes[0,1].plot(t, df["Actual Y"], label="gt y")

axes[1,0].plot(t, df["Roll"], label="roll")
axes[1,0].plot(t, df["Actual Roll"], label="gt roll")

axes[1,1].plot(t, df["Pitch"], label="pitch")
axes[1,1].plot(t, df["Actual Pitch"], label="gt pitch")

axes[0,0].set_title("X Offset")
axes[0,1].set_title("Y Offset")
axes[1,0].set_title("Roll")
axes[1,1].set_title("Pitch")

for ax in axes.flat:

    ax.legend()
    ax.grid()
plt.tight_layout()





In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(df["Total Offset X"], df["Total Offset Y"], label="trajectory")
plt.plot(df["Actual X"], df["Actual Y"], label="GT trajectory")

plt.plot(df["Total Offset X"].iloc[0], df["Total Offset Y"].iloc[0], "gP", label="start", markersize=17)


plt.plot(df["Total Offset X"].iloc[-1], df["Total Offset Y"].iloc[-1], "bs", label="end point", markersize=17)
plt.plot(df["Actual X"].iloc[-1], df["Actual Y"].iloc[-1], "rs", label="GT end point", markersize=17)


plt.xlabel("East origin offset, m", fontsize=18)
plt.ylabel("North origin offset, m", fontsize=18)

plt.xticks(np.arange(-0, 21, 2), fontsize=18)
plt.yticks(np.arange(30, 101, 20), fontsize=18)

plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.125),
          fancybox=True, shadow=True, ncol=5, fontsize=18)
plt.grid()
# plt.tight_layout()

plt.savefig("trajectory_kf.pdf", bbox_inches='tight')

In [ ]:
l2_err = np.sqrt((df["Total Offset X"] - df["Actual X"])**2 + (df["Total Offset Y"] - df["Actual Y"])**2)
plt.figure()
plt.plot(t, l2_err, label="L2 Error")
plt.title("L2 Error")
plt.xlabel("Time (s)")
plt.ylabel("L2 Error")
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()

print("L2 Final Error: ", l2_err.iloc[-1])
print("L2 Mean Error: ", l2_err.mean())

In [ ]:
def ang_err(a, b):
    p_a = np.array([np.cos(a), np.sin(a)])
    p_b = np.array([np.cos(b), np.sin(b)])
    return (np.linalg.norm(p_a - p_b) / 2)*100

l2_ang_err_roll = np.zeros(len(df))
l2_ang_err_pitch = np.zeros(len(df))
for i in range(len(df)):
    l2_ang_err_roll[i] = ang_err(df["Roll"][i], -df["Actual Roll"][i])
    l2_ang_err_pitch[i] = ang_err(df["Pitch"][i], -df["Actual Pitch"][i])
plt.figure()
plt.plot(t, l2_ang_err_roll, label="L2 Angle Error Percentage")
plt.plot(t, l2_ang_err_pitch, label="L2 Angle Error Pitch Percentage")

# plt.title("Angle Error")
plt.xlabel("Time (s)")
plt.ylabel("Angle Error, %")
plt.legend()

print("L2 Final Angle Error Roll: ", l2_ang_err_roll[-1])
print("L2 Final Angle Error Pitch: ", l2_ang_err_pitch[-1])
print("L2 Mean Angle Error Roll: ", l2_ang_err_roll.mean())
print("L2 Mean Angle Error Pitch: ", l2_ang_err_pitch.mean())



In [ ]:
x_gt = df["Actual X"].to_numpy()
vel_x = np.diff(x_gt)

y_gt = df["Actual Y"].to_numpy()
vel_y = np.diff(y_gt)


dt = 0.1
total_distance = np.cumsum(np.sqrt(vel_x**2 + vel_y**2) )
plt.figure()
plt.plot(t[1:], total_distance)
plt.title("Total Distance")
plt.xlabel("Time (s)")
plt.ylabel("Distance (m)")
plt.grid()
plt.show()
print("Total Distance: ", total_distance[-1])

In [ ]:
avg_vel = total_distance[-1] / t[-1]
print("Average Velocity: ", avg_vel)



In [ ]:
print(max(df["Actual Roll"]))
print(max(df["Actual Pitch"]))
# print(np.mean(df["Altitudee"]))


In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(13, 4))

fps = 10
period = 1/fps

t = np.arange(0, len(df)/fps, period)
print(t[-1])



axes.plot(t, df["Total Offset Y"], label="Estimated Offset")
axes.plot(t, df["Actual Y"], label="GT Offset")

axes.set_xlabel("Time, s", fontsize=18)
axes.set_ylabel("North Offset, m", fontsize=18)
axes.set_xlim(-1, t[-1]+1)
axes.set_xticks(np.arange(0, t[-1]+1, 10))
axes.grid()

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

# plt.tight_layout()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.175),
          fancybox=True, shadow=True, ncol=5, fontsize=18)

plt.savefig("north1.pdf", bbox_inches='tight')


In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(13, 4))


axes.plot(t, df["Total Offset X"], label="Estimated Offset")
axes.plot(t, df["Actual X"], label="GT Offset")
axes.set_xlim(-1, t[-1]+1)
axes.set_xticks(np.arange(0, t[-1]+1, 10))
axes.grid()


axes.set_xlabel("Time, s", fontsize=18)
axes.set_ylabel("East Offset, m", fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

# plt.tight_layout()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.175),
          fancybox=True, shadow=True, ncol=5, fontsize=18)
plt.savefig("east1.pdf", bbox_inches='tight')


In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(13, 4))


axes.plot(t, df["Roll"], label="Estimated Roll")
axes.plot(t, df["Actual Roll"], label="GT Roll")
axes.set_xlim(-1, t[-1]+1)
axes.set_xticks(np.arange(0, t[-1]+1, 5))
axes.grid()


axes.set_xlabel("Time, s", fontsize=18)
axes.set_ylabel("Roll, rad", fontsize=18)

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

# plt.tight_layout()
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.175),
          fancybox=True, shadow=True, ncol=5, fontsize=18)
plt.savefig("roll.pdf", bbox_inches='tight')